In [2]:
using JuMP
using CSV
using DataFrames
using AmplNLWriter

### Prepare input data
data_mass = dropmissing(CSV.read("input_data/plecak.csv"))
data_plan = CSV.read("input_data/plan.csv")

plan_summary = by(data_plan, [:Przedmiot], plan_summary -> size(plan_summary, 1))
mass_summary = by(data_mass, [:Przedmiot], mass_summary -> sum(mass_summary[:Waga]))

przybory_mass = mass_summary[mass_summary[:Przedmiot] .== "Przybory", :][:x1] / 1e3

mass_summary = mass_summary[mass_summary[:Przedmiot] .!= "Przybory", :]

dict_plan = Dict(zip(plan_summary[:Przedmiot], plan_summary[:x1]))
dict_mass = Dict(zip(mass_summary[:Przedmiot], mass_summary[:x1]))

data_in= DataFrame(Przedmiot = String[],
    Sztuk = Int64[],
    Waga = Float64[])

for row in 1:size(plan_summary, 1)
    course = plan_summary[:Przedmiot][row]
    push!(data_in,[course, get(dict_plan, course, 0), get(dict_mass, course, 0) / 1e3])
end

### set parameters
course_no = size(data_in,1)
max_course_occurrences = maximum(data_in[:Sztuk])
days = 5

# avg mass for comparison
avg_mass = sum(data_in[:Sztuk].*data_in[:Waga]) / days + przybory_mass

# objective scaling factor - workaround to set solution tolerance
objective_scaling_factor = 20e-3
data_in[:Waga] = data_in[:Waga] * objective_scaling_factor

# dictionaries for results
course_map = Dict(zip(1:course_no, data_in[:Przedmiot]))
mass_map = Dict(zip(1:course_no, data_in[:Waga]))

# maximum number of a course occurences per day
max_occurences_per_day  = 2
if max_occurences_per_day < max_course_occurrences
    max_course_occurrences = max_occurences_per_day
end

### Opt start, define model, variables, constraints and objective
# Solver engine can be found on from https://ampl.com/products/solvers/open-source/ 
solver_dir="couenne-win64/couenne.exe"
mdl=Model(solver=AmplNLSolver(solver_dir,["tol=1"]))

@variable(mdl, x[1:days,1:course_no, 1:max_course_occurrences], Bin, start=1)
@variable(mdl, day[1:days])

for d in 1:days
   @constraint(mdl, day[d] == sum(sum(x[d,c,m]*data_in[:Waga][c] 
                for c in 1:course_no) 
                for m in 1:max_course_occurrences))
    
    for c in 1:course_no
            @constraint(mdl, sum(x[d,c,m]*m 
                    for m in 1:max_course_occurrences) <= max_course_occurrences)
    end
end

for c in 1:course_no
    @constraint(mdl, sum(sum(x[d,c,m]*m 
                for d in 1:days) 
                for m in 1:max_course_occurrences) == data_in[:Sztuk][c])
end

@NLobjective(mdl,Min, sum((day[d])^2 for d in 1:days))

print(mdl)
# solve
status = solve(mdl)



Min day[1] ^ 2.0 + day[2] ^ 2.0 + day[3] ^ 2.0 + day[4] ^ 2.0 + day[5] ^ 2.0
Subject to
 day[1] - 0.01394 x[1,1,1] - 0.0143 x[1,2,1] - 0.03352 x[1,3,1] - 0.00594 x[1,4,1] - 0.011359999999999999 x[1,5,1] - 0.02428 x[1,6,1] - 0.01512 x[1,7,1] - 0.00318 x[1,8,1] - 0.00334 x[1,9,1] - 0.00968 x[1,10,1] - 0.0072 x[1,11,1] - 0.01394 x[1,1,2] - 0.0143 x[1,2,2] - 0.03352 x[1,3,2] - 0.00594 x[1,4,2] - 0.011359999999999999 x[1,5,2] - 0.02428 x[1,6,2] - 0.01512 x[1,7,2] - 0.00318 x[1,8,2] - 0.00334 x[1,9,2] - 0.00968 x[1,10,2] - 0.0072 x[1,11,2] == 0
 x[1,1,1] + 2 x[1,1,2] <= 2
 x[1,2,1] + 2 x[1,2,2] <= 2
 x[1,3,1] + 2 x[1,3,2] <= 2
 x[1,4,1] + 2 x[1,4,2] <= 2
 x[1,5,1] + 2 x[1,5,2] <= 2
 x[1,6,1] + 2 x[1,6,2] <= 2
 x[1,7,1] + 2 x[1,7,2] <= 2
 x[1,8,1] + 2 x[1,8,2] <= 2
 x[1,9,1] + 2 x[1,9,2] <= 2
 x[1,10,1] + 2 x[1,10,2] <= 2
 x[1,11,1] + 2 x[1,11,2] <= 2
 day[2] - 0.01394 x[2,1,1] - 0.0143 x[2,2,1] - 0.03352 x[2,3,1] - 0.00594 x[2,4,1] - 0.011359999999999999 x[2,5,1] - 0.02428 x[2,6,1] - 0.01512

Cbc0010I After 600 nodes, 216 on tree, 0.011837877 best solution, best possible 0.010237347 (1.57 seconds)
Cbc0010I After 700 nodes, 253 on tree, 0.011837877 best solution, best possible 0.010237568 (1.69 seconds)
Couenne: new cutoff value 1.1822578220e-002 (1.846 seconds)
Cbc0004I Integer solution of 0.011822675 found after 7100 iterations and 799 nodes (1.80 seconds)
Cbc0010I After 800 nodes, 163 on tree, 0.011822675 best solution, best possible 0.010237568 (1.80 seconds)
Cbc0010I After 900 nodes, 197 on tree, 0.011822675 best solution, best possible 0.01024071 (1.89 seconds)
Optimality Based BT: 9 improved bounds
Cbc0010I After 1000 nodes, 231 on tree, 0.011822675 best solution, best possible 0.010242148 (2.00 seconds)
Cbc0010I After 1100 nodes, 263 on tree, 0.011822675 best solution, best possible 0.010242625 (2.08 seconds)
Optimality Based BT: 7 improved bounds
Optimality Based BT: 9 improved bounds
Cbc0010I After 1200 nodes, 298 on tree, 0.011822675 best solution, best possible 0

Cbc0010I After 8000 nodes, 2095 on tree, 0.011822675 best solution, best possible 0.010243968 (6.82 seconds)
Cbc0010I After 8100 nodes, 2106 on tree, 0.011822675 best solution, best possible 0.010243968 (6.88 seconds)
Cbc0010I After 8200 nodes, 2132 on tree, 0.011822675 best solution, best possible 0.010243968 (6.93 seconds)
Cbc0010I After 8300 nodes, 2161 on tree, 0.011822675 best solution, best possible 0.010243968 (6.99 seconds)
Cbc0010I After 8400 nodes, 2186 on tree, 0.011822675 best solution, best possible 0.010243968 (7.05 seconds)
Cbc0010I After 8500 nodes, 2218 on tree, 0.011822675 best solution, best possible 0.010243968 (7.11 seconds)
Cbc0010I After 8600 nodes, 2245 on tree, 0.011822675 best solution, best possible 0.010243968 (7.19 seconds)
Cbc0010I After 8700 nodes, 2272 on tree, 0.011822675 best solution, best possible 0.010243968 (7.27 seconds)
Cbc0010I After 8800 nodes, 2298 on tree, 0.011822675 best solution, best possible 0.010243968 (7.33 seconds)
Cbc0010I After 8900

Cbc0010I After 15600 nodes, 3929 on tree, 0.011822675 best solution, best possible 0.010243993 (11.63 seconds)
Cbc0010I After 15700 nodes, 3964 on tree, 0.011822675 best solution, best possible 0.010243993 (11.68 seconds)
Cbc0010I After 15800 nodes, 3987 on tree, 0.011822675 best solution, best possible 0.010243993 (11.72 seconds)
Cbc0010I After 15900 nodes, 4008 on tree, 0.011822675 best solution, best possible 0.010243993 (11.79 seconds)
Cbc0010I After 16000 nodes, 4027 on tree, 0.011822675 best solution, best possible 0.010243993 (11.84 seconds)
Cbc0010I After 16100 nodes, 4047 on tree, 0.011822675 best solution, best possible 0.010243993 (11.90 seconds)
Cbc0010I After 16200 nodes, 4057 on tree, 0.011822675 best solution, best possible 0.010243993 (11.95 seconds)
Cbc0010I After 16300 nodes, 4070 on tree, 0.011822675 best solution, best possible 0.010243993 (12.00 seconds)
Cbc0010I After 16400 nodes, 4094 on tree, 0.011822675 best solution, best possible 0.010243993 (12.06 seconds)
C

Cbc0010I After 23100 nodes, 4852 on tree, 0.011822675 best solution, best possible 0.010254144 (16.34 seconds)
Cbc0010I After 23200 nodes, 4863 on tree, 0.011822675 best solution, best possible 0.010254789 (16.39 seconds)
Cbc0010I After 23300 nodes, 4866 on tree, 0.011822675 best solution, best possible 0.010255126 (16.45 seconds)
Cbc0010I After 23400 nodes, 4867 on tree, 0.011822675 best solution, best possible 0.010255581 (16.50 seconds)
Cbc0010I After 23500 nodes, 4864 on tree, 0.011822675 best solution, best possible 0.010256392 (16.56 seconds)
Cbc0010I After 23600 nodes, 4881 on tree, 0.011822675 best solution, best possible 0.010256393 (16.63 seconds)
Cbc0010I After 23700 nodes, 4888 on tree, 0.011822675 best solution, best possible 0.010256514 (16.69 seconds)
Cbc0010I After 23800 nodes, 4898 on tree, 0.011822675 best solution, best possible 0.010256702 (16.75 seconds)
Cbc0010I After 23900 nodes, 4909 on tree, 0.011822675 best solution, best possible 0.010257307 (16.83 seconds)
C

...

Cbc0010I After 75100 nodes, 5985 on tree, 0.011822675 best solution, best possible 0.011804105 (48.99 seconds)
Cbc0010I After 75200 nodes, 5954 on tree, 0.011822675 best solution, best possible 0.011804164 (49.04 seconds)
Cbc0010I After 75300 nodes, 5920 on tree, 0.011822675 best solution, best possible 0.011804329 (49.09 seconds)
Cbc0010I After 75400 nodes, 5907 on tree, 0.011822675 best solution, best possible 0.011804358 (49.15 seconds)
Cbc0010I After 75500 nodes, 5906 on tree, 0.011822675 best solution, best possible 0.011804358 (49.20 seconds)
Cbc0010I After 75600 nodes, 5906 on tree, 0.011822675 best solution, best possible 0.011804358 (49.26 seconds)
Cbc0010I After 75700 nodes, 5903 on tree, 0.011822675 best solution, best possible 0.011804358 (49.32 seconds)
Cbc0010I After 75800 nodes, 5898 on tree, 0.011822675 best solution, best possible 0.011804358 (49.37 seconds)
Cbc0010I After 75900 nodes, 5896 on tree, 0.011822675 best solution, best possible 0.011804358 (49.42 seconds)
C

Cbc0010I After 82500 nodes, 5580 on tree, 0.011822675 best solution, best possible 0.01180628 (53.29 seconds)
Cbc0010I After 82600 nodes, 5560 on tree, 0.011822675 best solution, best possible 0.011806409 (53.35 seconds)
Cbc0010I After 82700 nodes, 5544 on tree, 0.011822675 best solution, best possible 0.011806476 (53.41 seconds)
Cbc0010I After 82800 nodes, 5522 on tree, 0.011822675 best solution, best possible 0.011806565 (53.46 seconds)
Cbc0010I After 82900 nodes, 5503 on tree, 0.011822675 best solution, best possible 0.011806659 (53.52 seconds)
Cbc0010I After 83000 nodes, 5489 on tree, 0.011822675 best solution, best possible 0.011806787 (53.57 seconds)
Cbc0010I After 83100 nodes, 5459 on tree, 0.011822675 best solution, best possible 0.011806918 (53.62 seconds)
Cbc0010I After 83200 nodes, 5441 on tree, 0.011822675 best solution, best possible 0.011807002 (53.67 seconds)
Cbc0010I After 83300 nodes, 5423 on tree, 0.011822675 best solution, best possible 0.011807056 (53.72 seconds)
Cb

Cbc0010I After 89900 nodes, 4462 on tree, 0.011822675 best solution, best possible 0.011809618 (57.42 seconds)
Cbc0010I After 90000 nodes, 4446 on tree, 0.011822675 best solution, best possible 0.011809618 (57.49 seconds)
Cbc0010I After 90100 nodes, 4434 on tree, 0.011822675 best solution, best possible 0.011809618 (57.56 seconds)
Cbc0010I After 90200 nodes, 4414 on tree, 0.011822675 best solution, best possible 0.011809618 (57.61 seconds)
Cbc0010I After 90300 nodes, 4391 on tree, 0.011822675 best solution, best possible 0.011809618 (57.68 seconds)
Cbc0010I After 90400 nodes, 4377 on tree, 0.011822675 best solution, best possible 0.011809618 (57.74 seconds)
Cbc0010I After 90500 nodes, 4360 on tree, 0.011822675 best solution, best possible 0.011809652 (57.80 seconds)
Cbc0010I After 90600 nodes, 4339 on tree, 0.011822675 best solution, best possible 0.011809696 (57.86 seconds)
Cbc0010I After 90700 nodes, 4326 on tree, 0.011822675 best solution, best possible 0.011809699 (57.92 seconds)
C

Cbc0010I After 97300 nodes, 2082 on tree, 0.011822675 best solution, best possible 0.011810938 (61.72 seconds)
Cbc0010I After 97400 nodes, 2043 on tree, 0.011822675 best solution, best possible 0.011810955 (61.77 seconds)
Cbc0010I After 97500 nodes, 2001 on tree, 0.011822675 best solution, best possible 0.011811041 (61.83 seconds)
Cbc0010I After 97600 nodes, 1954 on tree, 0.011822675 best solution, best possible 0.011811116 (61.89 seconds)
Cbc0010I After 97700 nodes, 1904 on tree, 0.011822675 best solution, best possible 0.011811174 (61.95 seconds)
Cbc0010I After 97800 nodes, 1854 on tree, 0.011822675 best solution, best possible 0.011811244 (62.01 seconds)
Cbc0010I After 97900 nodes, 1804 on tree, 0.011822675 best solution, best possible 0.011811272 (62.06 seconds)
Cbc0010I After 98000 nodes, 1781 on tree, 0.011822675 best solution, best possible 0.011811302 (62.12 seconds)
Cbc0010I After 98100 nodes, 1732 on tree, 0.011822675 best solution, best possible 0.011811393 (62.17 seconds)
C

:Optimal

In [3]:
### print and save results
day_of_week=Dict(zip([1, 2, 3, 4, 5], ["Poniedzialek", "Wtorek", "Sroda", "Czwartek", "Piatek"]))

plan = DataFrame([String,String,String,String,String],
    [:Poniedzialek,:Wtorek,:Sroda,:Czwartek,:Piatek], 10)

for d in 1:days
    i=1
    plan[i,d] = string(getvalue(day[d]) / objective_scaling_factor + przybory_mass)
    i+=1
    for c in 1:course_no
        for m in 1:max_course_occurrences
            value = getvalue(x[d,c,m])*m
            if abs(value-0.0)>1e-3              
                for repeat in 1:m
                    plan[i,d] = get(course_map, c, "brak kursu w slowniku")
                    i+=1
                end
            end
        end
    end
    for rest in i:10
         plan[rest,d] = ""
    end
end

max_mass = maximum(getvalue(day)) / objective_scaling_factor + przybory_mass


println("masa maxymalna")
println(max_mass)
println("masa srednia bez optim")
println(avg_mass)
print(plan)

CSV.write("results/plan_wynik_min.csv",plan)

masa maxymalna
[3.907]
masa srednia bez optim
[5.3538]
10×5 DataFrames.DataFrame
│ Row │ Poniedzialek │ Wtorek    │ Sroda      │ Czwartek   │ Piatek   │
├─────┼──────────────┼───────────┼────────────┼────────────┼──────────┤
│ 1   │ [3.907]      │ [3.806]   │ [3.788]    │ [3.859]    │ [3.869]  │
│ 2   │ Polski       │ Angielski │ Matematyka │ Matematyka │ Przyroda │
│ 3   │ Polski       │ Polski    │ Matematyka │ Matematyka │ Przyroda │
│ 4   │ WF           │ Polski    │ Polski     │ Angielski  │ Technika │
│ 5   │ WF           │           │            │ Angielski  │ Historia │
│ 6   │ Informatyka  │           │            │ WF         │          │
│ 7   │ Muzyka       │           │            │ Religia    │          │
│ 8   │              │           │            │ Religia    │          │
│ 9   │              │           │            │ Plastyka   │          │
│ 10  │              │           │            │            │          │

CSV.Sink{Void,DataType}(    CSV.Options:
        delim: ','
        quotechar: '"'
        escapechar: '\\'
        missingstring: ""
        dateformat: nothing
        decimal: '.'
        truestring: 'true'
        falsestring: 'false'
        internstrings: true, IOBuffer(data=UInt8[...], readable=true, writable=true, seekable=true, append=false, size=0, maxsize=Inf, ptr=1, mark=-1), "results/plan_wynik_min.csv", 42, true, String["Poniedzialek", "Wtorek", "Sroda", "Czwartek", "Piatek"], 5, false, Val{false})